# 概要
* llama_indexのPromptHelperを触ってみる

# 参考
* トークンの制約に対応するための汎用プロンプトサポートツール
* https://gpt-index.readthedocs.io/en/latest/reference/prompt_helper.html

In [1]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
#Jupyterでも実行できるのは知らんかった...
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

# PromptHelperとは
* トークンの制約に対応するための汎用プロンプトサポートツール

# LLM（Large Language Model）
* PromptHelperの引数の大部分はLLMに関連する
* つまり最初にLLMが何かを理解する必要がある
* (そもそも入力されたPromptをparseするのはLLMなのでPromptHelperがLLMを引数にとるのは当然である)
* 詳細はここを見よ！
* https://gpt-index.readthedocs.io/en/latest/reference/llm_predictor.html

# そもそも
* LLMPredictorとはLangchainのLLMChainのラッパーであり、LlamdaIndexと統合することができる
* LLMChain ↔ LLMPrecitor ↔ LlamdaIndex
* Our LLMPredictor is a wrapper around Langchain’s LLMChain that allows easy integration into LlamaIndex.
* llmを引数にとり、デフォルトではOpenAI’s text-davinci-003が指定される
* max_input_size: int = 3900, num_output: int = 256
* デフォルトでは入力は3900トークンで出力は256トークン
* つまり長文の回答はできない

# 引数のllm
* OpenAI以外にいも複数のLLMをサポートしている
* OpenAI, Cohere, Hugging Face, etc
* Llama_indexは自身でLLMはサポートしておらず、あくまでもlangchainを介して呼び出しを行う？

In [3]:
from gpt_index import LLMPredictor, GPTSimpleVectorIndex, SimpleDirectoryReader

In [4]:
# OpenAIのAPIのラッパー
from langchain.llms import OpenAI

# 利用例

In [10]:
prompt = """
「オントロジー」という用語は、哲学が起源であるが、多くの異なる意味で用いられる。単語『オントロジー』は、ギリシャ語で「存在」を意味する ὄν (on) から来ている。計算機科学では、タイプ、特性、および関係タイプのセットから成る世界を記述するためのモデルを指すことが多い。正確にこれらの変化の周りに準備されるものは様々だが、それらはオントロジーの必需品である。実世界と一つのオントロジーでのモデルの特徴との間の近い類似性が一般に期待されている[3]。 多くのオントロジーが計算機科学と哲学の両方で共通に持っているものは、エンティティ、アイデア、およびイベントに関する特性と関係とともに、それらをある分類システムに沿って表現することである。両方のフィールドで、存在論的相対性の問題（例えば、哲学ではQuineとKripke、計算機科学ではSowa（英語版）とGuarino（英語版））において重要な作業を見つけ出し、そして規範的（英語版）オントロジーが実行可能かどうかに関わる討議をする（例えば、哲学における基本[4]を議論し、AIにおけるCycプロジェクトを議論する）ことである。2つの間の相違は主として焦点の問題である。計算機科学者が第一次原則の議論（そのようなものが固定化された本質として存在するかどうか、あるいはエンティティがプロセスよりオントロジー的により一次的でなければならないかどうかのような議論）にあまり関わらない一方で、哲学者は計算機科学における研究者より固定化され統制された語彙を確立することにそれほど関心がない。哲学者は人間のための思考道具としてオントロジーを用いるのに対して、計算機科学者は計算機処理を考えるために人間が使う思考道具としてオントロジーを用いたいという立ち位置の違いによる。
"""

In [11]:
res = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    temperature=0,
    max_tokens=4096
)

print(res)
print(res.choices[0].text)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"model": "text-davinci-003", "prompt": "\\n\\u300c\\u30aa\\u30f3\\u30c8\\u30ed\\u30b8\\u30fc\\u300d\\u3068\\u3044\\u3046\\u7528\\u8a9e\\u306f\\u3001\\u54f2\\u5b66\\u304c\\u8d77\\u6e90\\u3067\\u3042\\u308b\\u304c\\u3001\\u591a\\u304f\\u306e\\u7570\\u306a\\u308b\\u610f\\u5473\\u3067\\u7528\\u3044\\u3089\\u308c\\u308b\\u3002\\u5358\\u8a9e\\u300e\\u30aa\\u30f3\\u30c8\\u30ed\\u30b8\\u30fc\\u300f\\u306f\\u3001\\u30ae\\u30ea\\u30b7\\u30e3\\u8a9e\\u3067\\u300c\\u5b58\\u5728\\u300d\\u3092\\u610f\\u5473\\u3059\\u308b \\u1f44\\u03bd (on) \\u304b\\u3089\\u6765\\u3066\\u3044\\u308b\\u3002\\u8a08\\u7b97\\u6a5f\\u79d1\\u5b66\\u3067\\u306f\\u3001\\u30bf\\u30a4\\u30d7\\u3001\\u7279\\u6027\\u3001\\u304a\\u3088\\u3073\\u95a2\\u4fc2\\u30bf\\u30a4\\u30d7\\u306e\\u30bb\\u30c3\\u30c8\\u304b\\u3089\\u6210\\u308b\\u4e16\\u754c\\u3092\\u8a18\\u8ff0\\u3059\\u308b\\u305f\\u3081\

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 5172 tokens (1076 in your prompt; 4096 for the completion). Please reduce your prompt; or completion length.

In [9]:
prompt = """
初音ミクは
"""

res = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    max_tokens=200,
    temperature=0
)

print(res)
print(res.choices[0].text)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"model": "text-davinci-003", "prompt": "\\n\\u521d\\u97f3\\u30df\\u30af\\u306f\\n", "max_tokens": 200, "temperature": 0}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=9351 request_id=96da5bb0499195e384a66d1e4d5abd2d response_code=200
{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\u521d\u97f3\u30df\u30af\u306f\u3001\u65e5\u672c\u306eVOCALOID\uff08\u30dc\u30fc\u30ab\u30ed\u30a4\u30c9\uff09\u30b7\u30ea\u30fc\u30ba\u306e\u30ad\u30e3\u30e9\u30af\u30bf\u30fc\u3067\u3059\u30022007\u5e74\u306bCrypton Future Media\u304b\u3089\u767a\u58f2\u3055\u308c\u305f\u521d\u306eVOCALOID\u3068\u3057\u3066\u767b\u5834\u3057\u3001\u4ee

In [34]:
import openai

def text_summary(prompt):
    # 分析の実施
    response = openai.Completion.create(
    engine="text-davinci-002",
    prompt=prompt,
    temperature=0.7,
    max_tokens=500,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    )

    # 分析結果の出力
    return response["choices"][0]["text"].replace('\n','')

def crean_text(text):
    text= text.replace('　',' ')

    return text


# 要約対象の文章をここに入力
text="""
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。天下一武道会終了後、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。闘いの後、悟空はピッコロ大魔王に殺された神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。そこで神龍復活の条件として、神の下で天界で修行することとなった。その約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少年時代に出会った際に「嫁にもらう」と約束していた牛魔王の娘・チチと再会しその場で結婚。
"""

# 要約を3点で出力するように調整
prompt ='''
Tl;dr
要約は以下の通りです
'''

In [35]:
prompt = crean_text(text) +  prompt


# 結果出力
print(text_summary(prompt))

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-davinci-002/completions
DEBUG:openai:api_version=None data='{"prompt": "\\n\\u5730\\u7403\\u306e\\u4eba\\u91cc\\u96e2\\u308c\\u305f\\u5c71\\u5965\\u306b\\u4f4f\\u3080\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u306f\\u3042\\u308b\\u65e5\\u3001\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u6765\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3046\\u3002\\u305d\\u3053\\u3067\\u30017\\u3064\\u96c6\\u3081\\u308b\\u3068\\u795e\\u9f8d\\uff08\\u30b7\\u30a7\\u30f3\\u30ed\\u30f3\\uff09\\u304c\\u73fe\\u308c\\u3001\\u3069\\u3093\\u306a\\u9858\\u3044\\u3067\\u3082\\u4e00\\u3064\\u3060\\u3051\\u53f6\\u3048\\u3066\\u304f\\u308c\\u308b\\u3068\\u3044\\u3046\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u306e\\u5b58\\u5728\\u3092\\u3001\\u3055\\u3089\\u306b\\u80b2\\u3066\\u306e\\u89aa\\u3067\\u3042\\u308b\\u5b6b\\u609f\\u9

In [57]:
import openai

def text_summary(prompt):
    # 分析の実施
    response = openai.Completion.create(
    engine="text-davinci-002",
    prompt=prompt,
    temperature=0,
    max_tokens=2048,
    )

    # 分析結果の出力
    return response

def crean_text(text):
    text= text.replace('　',' ')

    return text


# 要約対象の文章をここに入力
text="""
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。天下一武道会終了後、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。闘いの後、悟空はピッコロ大魔王に殺された神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。そこで神龍復活の条件として、神の下で天界で修行することとなった。その約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少年時代に出会った際に「嫁にもらう」と約束していた牛魔王の娘・チチと再会しその場で結婚。そして武道会に出場していたピッコロ大魔王の生まれ変わりであるマジュニアと決勝戦で激突、これに勝利し初の天下一武道会優勝を飾る。ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空のもとに、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。さらわれた孫悟飯を助けるため悟空は宿敵ピッコロと手を組み、自らの命と引き換えにラディッツを倒すが、約1年後にはさらに強力なサイヤ人たちがドラゴンボールを求めて地球に来襲することを知る。悟空はドラゴンボールによって生き返るまでの間、あの世の界王の下で修業し、仲間と共に地球に強襲したサイヤ人の戦士・ナッパと王子・ベジータを迎え撃つ。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘。仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。
"""

# 要約を3点で出力するように調整
prompt ='''
Tl;dr
要約は以下の通りです
'''

In [58]:
prompt = crean_text(text) +  prompt


# 結果出力
print(text_summary(prompt))

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-davinci-002/completions
DEBUG:openai:api_version=None data='{"prompt": "\\n\\u5730\\u7403\\u306e\\u4eba\\u91cc\\u96e2\\u308c\\u305f\\u5c71\\u5965\\u306b\\u4f4f\\u3080\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u306f\\u3042\\u308b\\u65e5\\u3001\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u6765\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3046\\u3002\\u305d\\u3053\\u3067\\u30017\\u3064\\u96c6\\u3081\\u308b\\u3068\\u795e\\u9f8d\\uff08\\u30b7\\u30a7\\u30f3\\u30ed\\u30f3\\uff09\\u304c\\u73fe\\u308c\\u3001\\u3069\\u3093\\u306a\\u9858\\u3044\\u3067\\u3082\\u4e00\\u3064\\u3060\\u3051\\u53f6\\u3048\\u3066\\u304f\\u308c\\u308b\\u3068\\u3044\\u3046\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u306e\\u5b58\\u5728\\u3092\\u3001\\u3055\\u3089\\u306b\\u80b2\\u3066\\u306e\\u89aa\\u3067\\u3042\\u308b\\u5b6b\\u609f\\u9

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-davinci-002/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-davinci-002/completions processing_ms=16645 request_id=d160753236a2975418dad0faae3cd9a9 response_code=200
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\u30c9\u30e9\u30b4\u30f3\u30dc\u30fc\u30eb\u306f\u3001\u795e\u9f8d\uff08\u30b7\u30a7\u30f3\u30ed\u30f3\uff09\u304c\u73fe\u308c\u3001\u3069\u3093\u306a\u9858\u3044\u3067\u3082\u4e00\u3064\u3060\u3051\u53f6\u3048\u3066\u304f\u308c\u308b\u3068\u3044\u3046\u7403\u306e\u5b58\u5728\u3092\u77e5\u3063\u305f\u5c11\u5e74\u30fb\u5b6b\u609f\u7a7a\u304c\u3001\u30d6\u30eb\u30de\u3068\u5171\u306b\u6b8b\u308a\u306e\u30c9\u30e9\u30b4\u30f3\u30dc\u30fc\u30eb\u3092\u63a2\u3059\u65c5\u306b\u51fa\u308b\u3068\u3044\u3046\u7269\u8a9e\u3067\u3059\u3002\n\n\u30d4\u30c3\u30b3\u30ed\u5927\u9b

In [52]:
# 連続で実行するとtoken超過で失敗するっぽい
# まだ不安定なのか...

In [59]:
import openai

def text_summary(prompt):
    # 分析の実施
    response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=prompt,
    temperature=0,
    max_tokens=2048,
    )

    # 分析結果の出力
    return response

def crean_text(text):
    text= text.replace('　',' ')

    return text


# 要約対象の文章をここに入力
text="""
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。天下一武道会終了後、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。闘いの後、悟空はピッコロ大魔王に殺された神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。そこで神龍復活の条件として、神の下で天界で修行することとなった。その約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少年時代に出会った際に「嫁にもらう」と約束していた牛魔王の娘・チチと再会しその場で結婚。そして武道会に出場していたピッコロ大魔王の生まれ変わりであるマジュニアと決勝戦で激突、これに勝利し初の天下一武道会優勝を飾る。ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空のもとに、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。さらわれた孫悟飯を助けるため悟空は宿敵ピッコロと手を組み、自らの命と引き換えにラディッツを倒すが、約1年後にはさらに強力なサイヤ人たちがドラゴンボールを求めて地球に来襲することを知る。悟空はドラゴンボールによって生き返るまでの間、あの世の界王の下で修業し、仲間と共に地球に強襲したサイヤ人の戦士・ナッパと王子・ベジータを迎え撃つ。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘。仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。
"""

# 要約を3点で出力するように調整
prompt ='''
Tl;dr
要約は以下の通りです
'''

In [60]:
prompt = crean_text(text) +  prompt


# 結果出力
print(text_summary(prompt))

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-davinci-003/completions
DEBUG:openai:api_version=None data='{"prompt": "\\n\\u5730\\u7403\\u306e\\u4eba\\u91cc\\u96e2\\u308c\\u305f\\u5c71\\u5965\\u306b\\u4f4f\\u3080\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u306f\\u3042\\u308b\\u65e5\\u3001\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u6765\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3046\\u3002\\u305d\\u3053\\u3067\\u30017\\u3064\\u96c6\\u3081\\u308b\\u3068\\u795e\\u9f8d\\uff08\\u30b7\\u30a7\\u30f3\\u30ed\\u30f3\\uff09\\u304c\\u73fe\\u308c\\u3001\\u3069\\u3093\\u306a\\u9858\\u3044\\u3067\\u3082\\u4e00\\u3064\\u3060\\u3051\\u53f6\\u3048\\u3066\\u304f\\u308c\\u308b\\u3068\\u3044\\u3046\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u306e\\u5b58\\u5728\\u3092\\u3001\\u3055\\u3089\\u306b\\u80b2\\u3066\\u306e\\u89aa\\u3067\\u3042\\u308b\\u5b6b\\u609f\\u9

DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-davinci-003/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-davinci-003/completions processing_ms=23611 request_id=9b0666c42bb83c17a6822f0e16c801ad response_code=200
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\u5b6b\u609f\u7a7a\u306f\u3001\u30c9\u30e9\u30b4\u30f3\u30dc\u30fc\u30eb\u30927\u3064\u96c6\u3081\u308b\u3053\u3068\u3067\u795e\u9f8d\u3092\u53ec\u559a\u3067\u304d\u308b\u3053\u3068\u3092\u77e5\u308a\u3001\u30d6\u30eb\u30de\u3068\u5171\u306b\u6b8b\u308a\u306e\u30c9\u30e9\u30b4\u30f3\u30dc\u30fc\u30eb\u3092\u63a2\u3059\u65c5\u306b\u51fa\u308b\u3002\u305d\u306e\u9053\u4e2d\u3067\u6b66\u8853\u306e\u9054\u4eba\u30fb\u4e80\u4ed9\u4eba\u306e\u4e0b\u3067\u4fee\u884c\u3092\u7a4d\u307f\u3001\u5929\u4e0

In [5]:
import openai

def text_summary(prompt):
    # 分析の実施
    response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=prompt,
    temperature=0,
    max_tokens=4096,
    )

    # 分析結果の出力
    return response

def crean_text(text):
    text= text.replace('　',' ')

    return text


# 要約対象の文章をここに入力
text="""
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。天下一武道会終了後、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。闘いの後、悟空はピッコロ大魔王に殺された神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。そこで神龍復活の条件として、神の下で天界で修行することとなった。その約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少年時代に出会った際に「嫁にもらう」と約束していた牛魔王の娘・チチと再会しその場で結婚。そして武道会に出場していたピッコロ大魔王の生まれ変わりであるマジュニアと決勝戦で激突、これに勝利し初の天下一武道会優勝を飾る。ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空のもとに、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。さらわれた孫悟飯を助けるため悟空は宿敵ピッコロと手を組み、自らの命と引き換えにラディッツを倒すが、約1年後にはさらに強力なサイヤ人たちがドラゴンボールを求めて地球に来襲することを知る。悟空はドラゴンボールによって生き返るまでの間、あの世の界王の下で修業し、仲間と共に地球に強襲したサイヤ人の戦士・ナッパと王子・ベジータを迎え撃つ。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘。仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。
"""
#text="""
#地球の神と殺された仲間たちを甦らせるため、重傷で入院中の悟空に代わり、悟飯、クリリン、ブルマの3人が神とピッコロの故郷であるナメック星へ向かう。だが、そこには地球で闘ったベジータや、界王すら畏怖する宇宙の帝王・フリーザとその一味が不老不死を求めて来襲し、ナメック星人を虐殺しながらドラゴンボールを略奪していた。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる。出し抜かれて願いが叶えられなかったフリーザは激怒し、一行は対決を強いられる。フリーザの持つ圧倒的な力の前にはベジータやピッコロ、悟空すら歯が立たず仲間たちが次々と命を落としていった。怒りを爆発させた悟空は伝説の戦士・超（スーパー）サイヤ人へと覚醒。フルパワーを解放したフリーザに勝利する。ポルンガによって地球に帰還した悟飯たちは復活したドラゴンボールによりサイヤ人やフリーザ一味に殺された人々を蘇生させた。一方の悟空も爆発するナメック星を辛くも脱出、ヤードラット星に漂着し一命を取り留めた。ナメック星での闘いから約1年後、密かに生き延びていたフリーザとその一味が地球を襲撃するが、謎の超サイヤ人によって撃退される。トランクスと名乗るその青年は、自分は未来からやってきたブルマとベジータの息子であることを悟空にだけ明かすと同時に、悟空は心臓病によって命を落とすこと、3年後に現れる2体の人造人間が絶望の未来をもたらすことを告げる。その後本当に心臓病によって危篤状態になるも、トランクスから渡された特効薬によって生還、来るべき日に備えて3年間各々に修行してその日を迎える悟空たちであったが、事態はトランクスが知っている歴史とは大きく違うものとなり、彼さえ知らなかった人造人間たちまで現れ、さらに究極の人造人間・セルが未来から出現。悟空らの想定を遥かに超えた戦士が続々と現れた人造人間17号と人造人間18号を吸収することで完全体となったセルは地球の命運を賭けた武道会「セルゲーム」の開催を全世界に宣言する。悟空らは天界にある1日で1年の修行が行えるも過酷な環境に晒される「精神と時の部屋」で修行し、強さを増してセルゲームに臨むが、悟空はこのセルとの闘いの中で地球を守るために命を落とす。だが、その遺志を受け継いだ息子・悟飯が超サイヤ人2へと覚醒、父・悟空の幻影と共にかめはめ波を放ちセルを撃破。セルゲームを制した悟飯たちは、ドラゴンボールによりセルに殺された人々を蘇生させるが、悟空は自分が悪人を引き寄せているという理由で生き返りを拒否し、あの世に残ることを選ぶ。セルゲームより約7年後、高校生に成長した悟飯が天下一武道会に出場することを知った悟空は、自らも出場するために占いババの力によって1日だけこの世に戻る。天下一武道会の最中、悟空たちは界王よりもさらに高位の存在である界王神から、恐ろしい力を持つ魔人ブウの封印が解かれようとしていることを知らされる。復活した魔人ブウにより悟飯やベジータが倒され、悟空はあの世に帰ったため、地球の命運は悟空の次男・孫悟天と少年トランクスの幼い二人に託される。一方、魔人ブウは様々な人間との出会いからより邪悪で強力な魔人へと変貌。悟天とトランクスが「フュージョン（融合）」して誕生した戦士・ゴテンクスや、潜在能力を解放し、パワーアップを遂げて帰ってきた悟飯らが応戦するが、戦士たちを次々と吸収し姿を変えていく魔人ブウに苦戦を強いられる。危機に陥った悟飯らを救うため現世に舞い戻った悟空とベジータは、界王神界で真の姿となった魔人ブウとの最終決戦に臨む。ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。それから10年後、悟空は孫のパンと共に天下一武道会に久しぶりに出場し、魔人ブウの生まれ変わりである少年・ウーブと出会う。悟空はウーブと共に見果てぬ強さを追い求めて修行に旅立ち、物語は幕を閉じる。
#"""

# 要約を3点で出力するように調整
prompt ='''
Tl;dr
要約は以下の通りです
'''

In [6]:
prompt

'\nTl;dr\n要約は以下の通りです\n'

In [7]:
crean_text(text) + prompt

'\n地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。天下一武道会終了後、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。闘いの後、悟空はピッコロ大魔王に殺された神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。そこで神龍復活の条件として、神の下で天界で修行することとなった。その約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少年時代に出会った際に「嫁にもらう」と約束していた牛魔王の娘・チチと再会しその場で結婚。そして武道会に出場していたピッコロ大魔王の生まれ変わりであるマジュニアと決勝戦で激突、これに勝利し初の天下一武道会優勝を飾る。ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空のもとに、実兄・ラディッツが宇宙より来襲し、自分

In [8]:
prompt = crean_text(text) +  prompt
result = text_summary(prompt)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-davinci-003/completions
DEBUG:openai:api_version=None data='{"prompt": "\\n\\u5730\\u7403\\u306e\\u4eba\\u91cc\\u96e2\\u308c\\u305f\\u5c71\\u5965\\u306b\\u4f4f\\u3080\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u306f\\u3042\\u308b\\u65e5\\u3001\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u6765\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3046\\u3002\\u305d\\u3053\\u3067\\u30017\\u3064\\u96c6\\u3081\\u308b\\u3068\\u795e\\u9f8d\\uff08\\u30b7\\u30a7\\u30f3\\u30ed\\u30f3\\uff09\\u304c\\u73fe\\u308c\\u3001\\u3069\\u3093\\u306a\\u9858\\u3044\\u3067\\u3082\\u4e00\\u3064\\u3060\\u3051\\u53f6\\u3048\\u3066\\u304f\\u308c\\u308b\\u3068\\u3044\\u3046\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u306e\\u5b58\\u5728\\u3092\\u3001\\u3055\\u3089\\u306b\\u80b2\\u3066\\u306e\\u89aa\\u3067\\u3042\\u308b\\u5b6b\\u609f\\u9

DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.openai.com:443
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-davinci-003/completions HTTP/1.1" 400 295
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-davinci-003/completions processing_ms=8 request_id=36daa57ad3f2a9a31490c99eac52dc9e response_code=400
INFO:openai:error_code=None error_message="This model's maximum context length is 4097 tokens, however you requested 5981 tokens (1885 in your prompt; 4096 for the completion). Please reduce your prompt; or completion length." error_param=None error_type=invalid_request_error message='OpenAI API error received' stream_error=False


InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 5981 tokens (1885 in your prompt; 4096 for the completion). Please reduce your prompt; or completion length.

In [ ]:
# そもそも論として連続実行すると必ず失敗することが判明

In [97]:
prompt

'\n地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。天下一武道会終了後、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。闘いの後、悟空はピッコロ大魔王に殺された神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。そこで神龍復活の条件として、神の下で天界で修行することとなった。その約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少年時代に出会った際に「嫁にもらう」と約束していた牛魔王の娘・チチと再会しその場で結婚。そして武道会に出場していたピッコロ大魔王の生まれ変わりであるマジュニアと決勝戦で激突、これに勝利し初の天下一武道会優勝を飾る。ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空のもとに、実兄・ラディッツが宇宙より来襲し、自分